In [18]:
# sample reader to read samples from /data/runs/samples
# and build each sample into a matrix

# Isaac Berez
# 14.01.23

from scipy.io import mmread
import os
import glob
import pandas as pd
import numpy as np
from pandas_ods_reader import read_ods
from copy import deepcopy
import pprint

In [5]:
#get meta data from ods file, only will need sample id, sex, group
analysis_dir = '/home/isaac/analysis/scRNA-seq/'
dimorph_sample_file = 'Dimorph_samples.ods'
dimorph_df = read_ods(analysis_dir+dimorph_sample_file)
dimorph_df.head()

,Serial_Number:int,Date_Captured:string,Species:string,Transcriptome:string,Strain:string,Project:string,Group,ChipID:string,SampleID:string,DonorID:string,...,Comments:string,cDNA_Lib_Ok:string,ngperul_cDNA:float64,Avesizebp_cDNAlib,Date,cDNAul,LIbConstructionComment,ngperul_seqlib:float64,lengthbp_seqlib:float64,Sample_Index:string
0,106.0,43993.0,Mm,Mm10,Cntnp-C57Bl/6,Cntnp_KO,Cntnp-KO-F,10X54,10X54_1,Cntnp-KO_2-9F,...,"Cntnp-KO female, showing Cntnp ""male"" ASD emot...",None,24.7,NaN,None,NaN,None,17.9,473.0,B3
1,107.0,43993.0,Mm,Mm10,Cntnp-C57Bl/6,Cntnp_KO,Cntnp-KO-F,10X54,10X54_2,Cntnp-KO_4-10F,...,"Cntnp-KO female, showing Cntnp ""male"" ASD emot...",None,37.7,NaN,None,NaN,None,20.0,471.0,D3
2,212.0,2021-03-22,Mm,Mm10,Cntnp-C57Bl/6,Cntnp_KO,Cntnp-KO-M,10x98,10x98_2,Cntnp-KO_2_M,...,"v3! Cntnap KO #2, M; behavioral testing previo...",None,39.7,1873.5,2021-04-04,5.0,Single index; scale down; PCR_14 cycles,17.4,479.2,H9
3,213.0,2021-03-22,Mm,Mm10,Cntnp-C57Bl/6,Cntnp_KO,Cntnp-KO-M,10x98,10x98_3,Cntnp-KO_3_M,...,"v3! Cntnap KO #3, M; behavioral testing previo...",None,45.5,1841.7,2021-04-04,5.0,Single index; scale down; PCR_14 cycles,21.9,426.4,A10
4,98.0,05/31/20,Mm,Mm10,C57Bl/6,Dimorph,Breeder-F,10X51,10X51_2,DI-B1-F,...,"Breeders B1, 8pups DOB 27.5 (P4). perfusion aC...",None,29.8,NaN,None,NaN,None,22.2,474.0,A2


In [6]:
#fill in missing data types
dimorph_df_1 = dimorph_df.rename(columns={"Group":"Group:string", 
                                          "Avesizebp_cDNAlib":"Avesizebp_cDNAlib:float64",
                                          "Date":"Date:string",
                                          "cDNAul":"cDNAul:float64",
                                          "LIbConstructionComment": "LIbConstructionComment:string"
                                         })
#list(dimorph_df_1.columns)

In [7]:
#create lists of meta data header and data types by splitting at colon in each column name
mdata_header = []
dtype = []
for d in dimorph_df_1:
    mdata_header.append(d.split(":")[0])
    dtype.append(d.split(":")[1])
    

In [9]:
#get and return sample id
samples = [x for x in list(dimorph_df['SampleID:string']) if x != None]


In [11]:
#create meta data dictionary from sample ids
meta_data_dict = dict.fromkeys(samples)

In [13]:
#initialize headers for each sample
mdata_header_dict = dict.fromkeys(mdata_header)

In [20]:
for i in range(len(meta_data_dict.keys())):
    for key,value in zip(mdata_header_dict.keys(),list(dimorph_df_1.loc[i])):
        mdata_header_dict.update({key:value})
    meta_data_dict.update({list(meta_data_dict.keys())[i]:deepcopy(mdata_header_dict)})



In [22]:
pprint.pprint(meta_data_dict)

{'10X35_1': {'Age': '8w',
             'Avesizebp_cDNAlib': nan,
             'Cell_Conc': 1000.0,
             'ChipID': '10X35',
             'Comments': 'Naive 8wo females, pooled, left hemisphere. 5%opti.',
             'Date': None,
             'Date_Captured': '2019-11-20',
             'DonorID': 'DI1,DI2',
             'Group': 'Naïve-F',
             'LIbConstructionComment': None,
             'Num_Pooled_Animals': 2.0,
             'PCR_Cycles': 13.0,
             'Project': 'Dimorph',
             'SampleID': '10X35_1',
             'Sample_Index': 'D9',
             'Serial_Number': 63.0,
             'Sex': 'F',
             'Species': 'Mm',
             'Strain': None,
             'Target_Num_Cells': 5000.0,
             'Tissue': 'MeA_L',
             'Transcriptome': 'Mm10',
             'cDNA_Lib_Ok': 'Y',
             'cDNAul': nan,
             'lengthbp_seqlib': 416.0,
             'ngperul_cDNA': 23.3,
             'ngperul_seqlib': 13.6},
 '10X35_2': {'Age': '8

In [80]:
type(dimorph_df['Transcriptome:string'][0])

str

In [15]:

sample_dir = '/data/runs/samples/'
dimorph_samples = samples

os.chdir(sample_dir)
cwd = os.getcwd()
#print (cwd)

#for loop would go here to loop over samples
dir = str((glob.glob(cwd+'/'+dimorph_samples[0]+
                 '/'+'*out*'+'/'+'outs'+
                 '/'+'filtered_feature_bc_matrix'))).replace('[','').replace(']','')[1:-1]

#print(cwd)
#print (type(dir))
#print (os.listdir(dir))

m = mmread(dir + '/' + 'matrix.mtx')


In [16]:
m_arr = m.toarray()
print (m_arr)
print (m_arr.shape)
print (type(m_arr))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(27998, 6229)
<class 'numpy.ndarray'>


In [71]:
barcodes = pd.read_csv(dir+'/'+'barcodes.tsv', sep='\t',header = None)
print(type(barcodes))
barcodes

<class 'pandas.core.frame.DataFrame'>


,0
0,AAACCCACAACAGTGG-1
1,AAACCCACATGGCCCA-1
2,AAACCCAGTCCCTGAG-1
3,AAACGAACACTACAGT-1
4,AAACGAATCCCAGCGA-1
...,...
6224,TTTGGTTTCATCTGTT-1
6225,TTTGGTTTCTGTGCTC-1
6226,TTTGTTGAGATGTTCC-1
6227,TTTGTTGCATAGCACT-1


In [76]:
cells = barcodes.iloc[:,0]
cells.shape[0]

6229

In [106]:
dimorph_samples[0]

'10X54_1'

In [122]:
cells = barcodes.iloc[:,0] + dimorph_samples[0]
#print (type(cells))
cells = cells.values.reshape(cells.shape[0],1)
#cells = [cells + dimorph_samples[0] for x in cells]
print (cells)
print (cells.shape)
print (type(cells))

[['AAACCCACAACAGTGG-110X54_1']
 ['AAACCCACATGGCCCA-110X54_1']
 ['AAACCCAGTCCCTGAG-110X54_1']
 ...
 ['TTTGTTGAGATGTTCC-110X54_1']
 ['TTTGTTGCATAGCACT-110X54_1']
 ['TTTGTTGTCCGCGGAT-110X54_1']]
(6229, 1)
<class 'numpy.ndarray'>


In [123]:
features = pd.read_csv(dir+'/'+'features.tsv', sep='\t', header=None)
print(features)
print(features.iloc[0:4,1])

                        0               1                2
0      ENSMUSG00000051951            Xkr4  Gene Expression
1      ENSMUSG00000089699          Gm1992  Gene Expression
2      ENSMUSG00000102343         Gm37381  Gene Expression
3      ENSMUSG00000025900             Rp1  Gene Expression
4      ENSMUSG00000109048             Rp1  Gene Expression
...                   ...             ...              ...
27993  ENSMUSG00000079808      AC168977.1  Gene Expression
27994  ENSMUSG00000095041            PISD  Gene Expression
27995  ENSMUSG00000063897           DHRSX  Gene Expression
27996  ENSMUSG00000096730        Vmn2r122  Gene Expression
27997  ENSMUSG00000095742  CAAA01147332.1  Gene Expression

[27998 rows x 3 columns]
0       Xkr4
1     Gm1992
2    Gm37381
3        Rp1
Name: 1, dtype: object


In [124]:
gene_labels = features.iloc[:,1]
print (gene_labels)

gene_labels = gene_labels.values.reshape(gene_labels.shape[0],1)
#gene_labels = np.reshape(gene_labels ,(gene_labels.shape[0],1))
gene_labels.shape

0                  Xkr4
1                Gm1992
2               Gm37381
3                   Rp1
4                   Rp1
              ...      
27993        AC168977.1
27994              PISD
27995             DHRSX
27996          Vmn2r122
27997    CAAA01147332.1
Name: 1, Length: 27998, dtype: object


(27998, 1)

In [125]:
gene_labels

array([['Xkr4'],
       ['Gm1992'],
       ['Gm37381'],
       ...,
       ['DHRSX'],
       ['Vmn2r122'],
       ['CAAA01147332.1']], dtype=object)

In [126]:
#check for duplicates
print('unique values in gene labels: ', len(np.unique(gene_labels)))
print('total length gene labels: ', len(gene_labels))
print('# duplicates: ', len(gene_labels)-len(np.unique(gene_labels)))

unique values in gene labels:  27933
total length gene labels:  27998
# duplicates:  65


In [ ]:
#for duplicate 

In [128]:
len(cells)

6229

In [132]:
#combine into single dataframe
sample_df = pd.DataFrame(data = m_arr,
                         index = gene_labels,
                         columns = cells)
sample_df.iloc[:10,:4]

,"(AAACCCACAACAGTGG-110X54_1,)","(AAACCCACATGGCCCA-110X54_1,)","(AAACCCAGTCCCTGAG-110X54_1,)","(AAACGAACACTACAGT-110X54_1,)"
"(Xkr4,)",0,0,0,0
"(Gm1992,)",0,0,0,0
"(Gm37381,)",0,0,0,0
"(Rp1,)",0,0,0,0
"(Rp1,)",0,0,0,0
"(Sox17,)",0,0,0,0
"(Gm37323,)",0,0,0,0
"(Mrpl15,)",0,0,1,2
"(Lypla1,)",0,0,0,0
"(Gm37988,)",0,0,0,0


In [136]:
sample_df

,"(AAACCCACAACAGTGG-110X54_1,)","(AAACCCACATGGCCCA-110X54_1,)","(AAACCCAGTCCCTGAG-110X54_1,)","(AAACGAACACTACAGT-110X54_1,)","(AAACGAATCCCAGCGA-110X54_1,)","(AAACGAATCTGCTTAT-110X54_1,)","(AAACGCTAGCAGATAT-110X54_1,)","(AAACGCTAGTGCTCAT-110X54_1,)","(AAACGCTGTCCGTACG-110X54_1,)","(AAACGCTGTGGCTACC-110X54_1,)",...,"(TTTGGTTAGAGTCACG-110X54_1,)","(TTTGGTTCACAGCCTG-110X54_1,)","(TTTGGTTCAGCCTTCT-110X54_1,)","(TTTGGTTGTCACTAGT-110X54_1,)","(TTTGGTTGTTCCTAAG-110X54_1,)","(TTTGGTTTCATCTGTT-110X54_1,)","(TTTGGTTTCTGTGCTC-110X54_1,)","(TTTGTTGAGATGTTCC-110X54_1,)","(TTTGTTGCATAGCACT-110X54_1,)","(TTTGTTGTCCGCGGAT-110X54_1,)"
"(Xkr4,)",0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
"(Gm1992,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(Gm37381,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(Rp1,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(Rp1,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(AC168977.1,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(PISD,)",0,1,2,0,0,1,0,1,1,0,...,1,3,1,0,0,0,0,0,0,0
"(DHRSX,)",0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(Vmn2r122,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [135]:
#add duplicate rows together
sample_df_summed = sample_df.groupby(level=0).sum()
sample_df_summed

,"(AAACCCACAACAGTGG-110X54_1,)","(AAACCCACATGGCCCA-110X54_1,)","(AAACCCAGTCCCTGAG-110X54_1,)","(AAACGAACACTACAGT-110X54_1,)","(AAACGAATCCCAGCGA-110X54_1,)","(AAACGAATCTGCTTAT-110X54_1,)","(AAACGCTAGCAGATAT-110X54_1,)","(AAACGCTAGTGCTCAT-110X54_1,)","(AAACGCTGTCCGTACG-110X54_1,)","(AAACGCTGTGGCTACC-110X54_1,)",...,"(TTTGGTTAGAGTCACG-110X54_1,)","(TTTGGTTCACAGCCTG-110X54_1,)","(TTTGGTTCAGCCTTCT-110X54_1,)","(TTTGGTTGTCACTAGT-110X54_1,)","(TTTGGTTGTTCCTAAG-110X54_1,)","(TTTGGTTTCATCTGTT-110X54_1,)","(TTTGGTTTCTGTGCTC-110X54_1,)","(TTTGTTGAGATGTTCC-110X54_1,)","(TTTGTTGCATAGCACT-110X54_1,)","(TTTGTTGTCCGCGGAT-110X54_1,)"
"(0610007P14Rik,)",0,1,5,0,0,0,1,0,2,1,...,1,2,0,0,1,0,0,0,0,0
"(0610009B22Rik,)",0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
"(0610009L18Rik,)",0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(0610009O20Rik,)",0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
"(0610010F05Rik,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(mt-Nd3,)",1,9,17,21,9,6,14,23,10,10,...,16,10,4,1,3,4,4,8,14,1
"(mt-Nd4,)",10,60,117,66,30,50,87,144,80,53,...,82,76,44,12,9,38,10,57,73,24
"(mt-Nd4l,)",0,7,8,3,4,3,1,5,10,2,...,8,8,2,0,0,2,2,2,6,1
"(mt-Nd5,)",3,7,14,8,5,5,7,19,9,5,...,14,10,0,0,1,7,0,6,11,3
